# **GPT4ALL QA from docs:**
**Para analizar grandes cantidades de datos el precio de OpenAI deja de ser trivial, por lo tanto se explora la posibilidad de usar modelos en local que realicen lo mismo pero con el nivel más parecido al SotA actual.
Es por esto que se intenta usar [GPT4ALL](https://github.com/nomic-ai/gpt4all) y su implementación mediante librería [Langchain](https://python.langchain.com/en/latest/index.html#)**

**Sin embargo, los embeddings de OPENAI siguen siendo baratos y efectivos, por lo que su uso sí tiene sentido**

In [1]:
# Imports and setup
import numpy as np
import pandas as pd
import csv
import os
import re
import io
import json

import pypdf
import tabula

import openai
import pyllamacpp
import tiktoken

import langchain
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.callbacks.base import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.retrievers import SVMRetriever

api_key = json.load(open('./data/creds/gpt_id.json'))['api_key']
openai.api_key = api_key
os.environ['OPENAI_API_KEY'] = api_key

## Preprocesado Inicial:

In [2]:
# Funciones esenciales de procesamiento:
def limpieza_texto(texto: str) -> str:
    '''
    Función para limpiar texto de pdfs.
    Cambia saltos de línea, espacios en blanco y caracteres especiales.
    '''
    # Eliminamos espacios en blanco
    #texto = re.sub(' +', ' ', texto)
    # Eliminamos caracteres especiales [REVISAR]
    #texto = re.sub('[^A-Za-z0-9]+', ' ', texto)
    # Eliminamos saltos múltiples de línea
    texto = re.sub(r"\n\s*\n", "\n\n", texto)
    texto = re.sub(r"\n", ";", texto)
    return texto

def tabla_a_texto(tabla):
    '''
    Función para convertir una tabla de pandas en un texto.
    La idea es identificar los nombres de columna e índices correctos y
    a partir de ahí generar un texto que pueda ser procesado por el modelo.
    '''
    tabla = tabla.copy()
    
    # Tamaño mínimo de tabla para que sea válida = 2x2
    if sum(tabla.shape) < 4:
        return ''
    
    # Lista de valores que consideramos NaN:
    nan_equivalents = [np.NaN, np.nan,
                       'nan', 'NaN', 'Nan', 'NAN', 'na', 'NA',
                       'Unnamed:0', 'Unnamed: 0'
                       '', '-', ' ', '  ', '   ']
    
    # Asumimos que el primer elemento es el título salvo si es NaN:
    titulo = tabla.columns[0] if tabla.columns[0] not in nan_equivalents else ''
    
    # Asumimos que la primera columna es el índice y la eliminamos:
    tabla.index = tabla[tabla.columns[0]].values
    tabla.drop(columns=tabla.columns[0], inplace=True)

    # Si las columnas tienen muchos 'Unnamed' suele ser porque hay
    # varias líneas de texto. En ese caso, las juntamos en una sola:
    if sum(['Unnamed' in i for i in tabla.columns]) > 2:
        nueva_columna = [f'{tabla.columns[i]} {tabla.iloc[0,i]}'
                         for i in range(len(tabla.columns))]
        nueva_columna = [i.replace('Unnamed: ','') for i in nueva_columna]
        tabla.columns = nueva_columna

    
    # Eliminamos las filas y columnas que no tienen datos:
    tabla.replace(nan_equivalents, np.nan, inplace=True)
    tabla.dropna(axis=0, how='all', inplace=True)
    tabla.dropna(axis=1, how='all', inplace=True)
    
    # Check si las columnas son años:
    col_años = False
    years_txt=[str(i) for i in range(2015,2022)]
    years_int=[i for i in range(2015,2022)]
    years = set(years_txt+years_int)
    cruce = set(tabla.columns).intersection(set(years))
    if len(cruce) > 0: col_años=True
    
    # Si no son años las columnas, buscamos filas que sean años:
    contexto = None
    if not col_años:
        for i in tabla.iterrows():
            #print(i[1].values)
            try:
                cruce = set(i[1].values).intersection(set(years))
            except:
                cruce=[]
            if len(cruce)>0: # Si encontramos una fila con años:
                # Asignamos los años a las columnas:
                tabla.columns = i[1].values
                try: 
                    contexto = i[1].name
                except:
                    contexto = None
                # Drop de la fila:
                tabla.drop(i[0], inplace=True)
                break
    # Los procesos anteriores pueden haber dejado filas vacías, las eliminamos:
    tabla.replace(nan_equivalents, np.nan, inplace=True)
    tabla.dropna(axis=0, how='all', inplace=True)
    tabla.dropna(axis=1, how='all', inplace=True)
    # Pasamos a texto:
    texto = ''
    for i in tabla.items():
        txt = [f' {titulo} + {i[0]} + {x[0]} = {x[1]}; '
            for x in list(i[1].items())]
        add= ''.join(txt)
        if contexto:
            txt = [f' {titulo} + {contexto} + {i[0]} + {x[0]} = {x[1]}; '
                for x in list(i[1].items())]
            add = ''.join(txt)
        add = add.replace('  ',' ').replace('\n','; ').replace('  ','')
        texto += f';  Tabla={titulo}: {add}'
    return texto

def extract_text_from_pdf(pdf_path: str) -> list:
    '''
    Función para extraer texto de un pdf y limpiarlo.
    Devuelve una lista de str, cada una es una página del pdf.
    '''
    # Abrimos el pdf
    with open(pdf_path, 'rb') as f:
        pdf = pypdf.PdfReader(f)
        # Obtenemos el número de páginas
        num_pags = len(pdf.pages)
        count = 0
        text = []
        # Iteramos sobre las páginas
        for pag in pdf.pages:
            count +=1
            texto_pagina = pag.extract_text()
            tablas = tabula.read_pdf(pdf_path, pages=count)
            for tabla in tablas:
                texto_pagina += f'; {tabla_a_texto(tabla=tabla)}; '
            texto_pagina = limpieza_texto(texto_pagina)
            text.append(texto_pagina)
    return text

In [3]:
# Procesamos el PDF:
resultado = extract_text_from_pdf('./data/BME_Instituto_Broschure_mIA-X_A4_CMYK-U_v07.pdf')

In [4]:
resultado_unido = ' '.join(resultado)

In [5]:
# Guardamos el resultado en un csv con 1 fila por página
with open('lista_test_2.csv', 'w', newline='') as file:
    
    writer = csv.writer(file)
    writer.writerows([[str(i)] for i in resultado])

# Y entero como txt:
with open('texto_test_2.txt', 'w', newline='') as file:
    file.write(resultado_unido)

## Preprocesado de Langchain:

In [6]:
# Cargamos el csv como una lista (para comprobar que se ha guardado bien):
csv_loaded = list(pd.read_csv('lista_test_2.csv', header=None)[0])


In [4]:
# SPLITTER ESTANDAR:
splitter = langchain.text_splitter.RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=20,
    length_function = len)

loader = CSVLoader(file_path='lista_test_2.csv')

documentos = splitter.create_documents(csv_loaded,
                          metadatas=[{'source': f'lista_test_2.csv pag{i}'} for i in list(range(len(csv_loaded)))])

In [7]:
# SPLITTER por Tokens - TikToken Tokenizer:
def contador_tokens(texto, tokenizador= tiktoken.get_encoding('cl100k_base')):
    return len(tokenizador.encode(texto, disallowed_special=()))

tk_splitter = langchain.text_splitter.RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=35,
    length_function = contador_tokens,
    separators = ['\n','\r','\t',' ','\n\n'])

documentos = tk_splitter.create_documents(
    resultado, 
    metadatas=[{'source': f'lista_test_2.csv pag.{i}'}
               for i in list(range(len(csv_loaded)))]
    )

In [35]:
len(documentos)

394

In [8]:
embedding = langchain.embeddings.openai.OpenAIEmbeddings()
vector_store = langchain.vectorstores.Chroma.from_documents(documentos, embedding, persist_directory='./data/vector_stores')

Using embedded DuckDB with persistence: data will be stored in: ./data/vector_stores


In [37]:
vector_store.add_documents(documentos)

['1bd6471b-daf8-11ed-8cd3-6c6a77e568a6',
 '1bd6471c-daf8-11ed-994b-6c6a77e568a6',
 '1bd6471d-daf8-11ed-a658-6c6a77e568a6',
 '1bd6471e-daf8-11ed-a749-6c6a77e568a6',
 '1bd6471f-daf8-11ed-a208-6c6a77e568a6',
 '1bd64720-daf8-11ed-b111-6c6a77e568a6',
 '1bd64721-daf8-11ed-b617-6c6a77e568a6',
 '1bd64722-daf8-11ed-b74a-6c6a77e568a6',
 '1bd64723-daf8-11ed-aea7-6c6a77e568a6',
 '1bd64724-daf8-11ed-80fb-6c6a77e568a6',
 '1bd64725-daf8-11ed-b578-6c6a77e568a6',
 '1bd64726-daf8-11ed-97c4-6c6a77e568a6',
 '1bd64727-daf8-11ed-a0b0-6c6a77e568a6',
 '1bd64728-daf8-11ed-bf9b-6c6a77e568a6',
 '1bd64729-daf8-11ed-9de3-6c6a77e568a6',
 '1bd6472a-daf8-11ed-a45f-6c6a77e568a6',
 '1bd6472b-daf8-11ed-94e4-6c6a77e568a6',
 '1bd6472c-daf8-11ed-b4b7-6c6a77e568a6',
 '1bd6472d-daf8-11ed-8339-6c6a77e568a6',
 '1bd6472e-daf8-11ed-8566-6c6a77e568a6',
 '1bd6472f-daf8-11ed-a221-6c6a77e568a6',
 '1bd64730-daf8-11ed-b653-6c6a77e568a6',
 '1bd64731-daf8-11ed-9ef0-6c6a77e568a6',
 '1bd64732-daf8-11ed-a32d-6c6a77e568a6',
 '1bd64733-daf8-

In [38]:
vector_store.persist()

## Tests:

In [10]:
# PREGUNTA
pregunta = 'What can you learn in this Master?'
similares = vector_store.similarity_search(pregunta, include_metadata=True, top_k=5)
print(similares)

[Document(page_content=';de conocimiento, te llevas compañeros del mismo gremio con los que te ;ayudarás siempre. Por todo ello, mi experiencia fue más que positiva. Este ;máster te proporciona ese rasgo diferenciador que tan demandado es ac -;tualmente por las empresas del sector. Sin duda, lo recomiendo.CELSO OTERO GARCÍA (6ª EDICIÓN);Responsable Departamento Inteligencia Artificial Renta 4 Banco;Llevo en torno a 20 años en el mundo financiero desempeñando dis tintas tar -;eas, análisis, gestión de fondos, desarrollo de productos, etc. En este período, ;la fo rma de desempeñar el trabajo ha sufrido una evolución constante. ;La capacidad de', metadata={'source': 'lista_test_2.csv pag.8'}), Document(page_content='de resaltar que este es un máster de intensidad ;elevada. En media, el 30% de los alumnos no consiguen ;terminarlo. Para adquirir los conocimientos en Progra -;mación, Bolsa, Big Data, Inteligencia Artificial, Blockchain ;y Computación Cuántica hay que estudiar a diario. A con

### Ejemplo de Cadena de OpenAI funcional:

In [18]:
vector_store

AttributeError: 'Chroma' object has no attribute 'embeddings'

In [22]:
recuperador = SVMRetriever.from_texts([i.page_content for i in documentos], embedding)

In [24]:
recuperador.get_relevant_documents(pregunta)

[Document(page_content=';de conocimiento, te llevas compañeros del mismo gremio con los que te ;ayudarás siempre. Por todo ello, mi experiencia fue más que positiva. Este ;máster te proporciona ese rasgo diferenciador que tan demandado es ac -;tualmente por las empresas del sector. Sin duda, lo recomiendo.CELSO OTERO GARCÍA (6ª EDICIÓN);Responsable Departamento Inteligencia Artificial Renta 4 Banco;Llevo en torno a 20 años en el mundo financiero desempeñando dis tintas tar -;eas, análisis, gestión de fondos, desarrollo de productos, etc. En este período, ;la fo rma de desempeñar el trabajo ha sufrido una evolución constante. ;La capacidad de', metadata={}),
 Document(page_content='de resaltar que este es un máster de intensidad ;elevada. En media, el 30% de los alumnos no consiguen ;terminarlo. Para adquirir los conocimientos en Progra -;mación, Bolsa, Big Data, Inteligencia Artificial, Blockchain ;y Computación Cuántica hay que estudiar a diario. A con -;tinuación, desglosamos la inte

In [25]:
# Definimos un modelo recuperador (busca en embeddings)
# recuperador = vector_store.as_retriever(
#     search_kwargs={'top_k': 5, 'include_metadata': True}
# )

# Definimos el modelo de OpenAI:
openaillm = langchain.llms.openai.OpenAI(
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))

# Cadena (orquestador de la respuesta)
cadena_openai = langchain.chains.RetrievalQA.from_chain_type(
    chain_type='stuff',
    retriever=recuperador,
    llm=openaillm,
    return_source_documents=True)

In [26]:
# Resultado con sources:
cadena_openai({'query': pregunta})

{'query': 'What can you learn in this Master?',
 'result': ' In this Master, you can learn Machine Learning, reinforcement learning, natural language processing, cloud computing, and more.',
 'source_documents': [Document(page_content=';de conocimiento, te llevas compañeros del mismo gremio con los que te ;ayudarás siempre. Por todo ello, mi experiencia fue más que positiva. Este ;máster te proporciona ese rasgo diferenciador que tan demandado es ac -;tualmente por las empresas del sector. Sin duda, lo recomiendo.CELSO OTERO GARCÍA (6ª EDICIÓN);Responsable Departamento Inteligencia Artificial Renta 4 Banco;Llevo en torno a 20 años en el mundo financiero desempeñando dis tintas tar -;eas, análisis, gestión de fondos, desarrollo de productos, etc. En este período, ;la fo rma de desempeñar el trabajo ha sufrido una evolución constante. ;La capacidad de', metadata={}),
  Document(page_content='de resaltar que este es un máster de intensidad ;elevada. En media, el 30% de los alumnos no cons

### Test con GPT4ALL:

In [43]:
gpt4all_path = './data/models/gpt4all-7B/gpt4all-lora-quantized_new.bin'
callback = CallbackManager([StreamingStdOutCallbackHandler()])

gpt4allm = langchain.llms.gpt4all.GPT4All(model=gpt4all_path,
                                          n_threads=12,
                                          callback_manager=callback,
                                          n_ctx=2000)

cadena_gpt4all = langchain.chains.RetrievalQA.from_chain_type(
    chain_type='stuff',
    retriever=recuperador,
    llm=gpt4allm,
    return_source_documents=True)

In [46]:
# Resultado con sources:
cadena_openai({'query': pregunta})

{'query': 'How much CO2 was emitted in total according to this document? The ammount usually is in Tonnes of CO2.',
 'result': ' According to the document, the total CO2 emitted was 8,871 tonnes.',
 'source_documents': [Document(page_content='Distr.  Gr Mad  ;y SSRR AH Distr. ;Tonnes of Scope 1 ;CO2 emitted 805 1,494 2,663 836 468 1,399 2,880 1,262 72% 7% -8% -34%;Tonnes of Scope ;2 CO2 emitted 0 0 0 102 1,101 2,245 2,565 179 -100% -100% -100% -43%;Tonnes of Scope ;2 CO2 avoided (*) 1,193 2,198 2,999 96 0 0 0 0 - - - -;Tonnes CO2/ / ;million units. 0.004 22.58 104.85 42.30 0.003 27.54 137.86 71.42 38% -18% -24% -41%305-1;91; ;  Tabla=:  + 1 Granada + nan = Granada;+ 1 Granada + CO2 emitted = 805;+ 1 Granada + 2 CO2 emitted = 0;+ 1', metadata={'source': 'lista_test_2.csv pag90'}),
  Document(page_content='7% -8% -34%;Tonnes of Scope ;2 CO2 emitted 0 0 0 102 1,101 2,245 2,565 179 -100% -100% -100% -43%;Tonnes of Scope ;2 CO2 avoided (*) 1,193 2,198 2,999 96 0 0 0 0 - - - -;Tonnes CO2/ / 

### GPT4All-J:

In [ ]:
gpt4all_j_path = './data/models/gpt4all-7B/ggml-gpt4all-j.bin'

gpt4alljm = langchain.llms.gpt4all.GPT4All(model=gpt4all_j_path,
                                          n_threads=12,
                                          callback_manager=callback,
                                          n_ctx=2000)

In [7]:
# Modelos:
gpt4all_path = './data/models/gpt4all-7B/gpt4all-lora-quantized_new.bin'
gpt4all_j_path = './data/models/gpt4all-7B/ggml-gpt4all-j.bin'

gpt4a = langchain.llms.gpt4all.GPT4All(model=gpt4all_path,
                                     n_threads=12,
                                     n_ctx=2000)

# gpt4aj = langchain.llms.gpt4all.GPT4All(model=gpt4all_j_path,
#                                      n_threads=12)

opai_llm = langchain.llms.openai.OpenAI(callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))


In [10]:
recuperador = vector_store.as_retriever()

In [ ]:
recuperador

In [17]:
# OPENAI:
cadena_OAI = langchain.chains.question_answering.load_qa_chain(llm=opai_llm)

# GPT4ALL:
cadena_GPT4all = langchain.chains.question_answering.load_qa_chain(llm=gpt4a)

# GPT4ALL Alternativa:
esquema_pregunta = '''
Find the answer to the following question:
Question: {question}
Using the following context:
{context}
Answer: '''
prompt = langchain.PromptTemplate(
    template = esquema_pregunta,
    input_variables=['question', 'context'],
)
cadena_alt = langchain.LLMChain(
    prompt=prompt,
    llm=gpt4a)

# GPT4ALL Alternativa2:

cadena_alt2 = langchain.chains.RetrievalQA.from_chain_type(llm=gpt4a,
                                                           chain_type='stuff',
                                                           retriever=recuperador)


In [12]:
cadena_alt2.run(pregunta)

KeyboardInterrupt: 

[Document(page_content='emitted = -100%;+ 6 AH + 2 CO2 avoided (*) = nan;+ 6 AH + million units. = -24%; ;  Tabla=:  + 7 Distr. + nan = Distr.;+ 7 Distr. + CO2 emitted = -34%;+ 7 Distr. + 2 CO2 emitted = -43%;+ 7 Distr. + 2 CO2 avoided (*) = nan;+ 7 Distr. + million units. = -41%; ;', metadata={'source': 'lista_test_2.csv pag90'})]


In [21]:
# OPENAI:
cadena_OAI.run(
    input_documents = similares,
    question = pregunta)  

' In this document, CO2 emission is calculated in percentages. The first entry states that -100% of CO2 was emitted, and the last entry states that -41% of CO2 was emitted when 7 Distr. and a million units were used. So, in total, the amount of CO2 emitted was -141%, which is equivalent to 1.41 times the amount of CO2 that would normally be emitted.'

In [14]:
# GPT4ALL:
cadena_GPT4all.run(
    input_documents = similares,
    question = pregunta)  

KeyboardInterrupt: 

In [15]:
# GPT4ALL con Cadena Alternativa:
contexto_pregunta = f"{' '.join(i.page_content for i in similares)}"
cadena_alt.run(
    question = pregunta,
    context = contexto_pregunta)

KeyboardInterrupt: 

In [16]:
len(contexto_pregunta)

178

In [67]:
similares

[Document(page_content='REPORT;20;20: 98www.rovi.es', metadata={'source': 'lista_test_2.csv', 'row': 96}),
 Document(page_content='REPORT;20;20: SHAREHOLDER COMPOSITION;63.11%;Norbel Inversores, S.L.;5.57%;Indumenta Pueri, S.L.;3.043%;T.Rowe Price International Funds, Inc;3.005%;Wellington Management Group, LLP;25.275%;Other;7', metadata={'source': 'lista_test_2.csv', 'row': 5}),
 Document(page_content='REPORT;20;20: –Profarma;Each year, in the Plan Profarma, the Ministry of Industry, Tourism and Trade and the Ministry ;of Health, Social Services and Equality classify the pharmaceutical Companies in accordance ;with their contribution to the Spanish industrial fabric, taking their investment in technology, ;new manufacturing plants, research efforts, etc. as a reference. In February 2020, the results ;of Plan Profarma 2019 were issued and ROVI obtained the classification of Excellent for the ;fourteenth consecutive year. ;KEY FIGURES;(million euros) 2020 2019 2018 2017;Total revenue 42

In [ ]:
pregunta = '¿De qué empresa es este informe?'
similares = recuperador.get_relevant_documents(pregunta)
cadena.run(input_documents = similares, question = pregunta)  

''